# Import libraries

In [75]:
from langchain.schema import SystemMessage
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.chat_models import ChatOpenAI
import glob
import pandas as pd
from datasets import Dataset, load_from_disk

import os
from dotenv import load_dotenv
import json
import tqdm

# Load env variables

In [7]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')
PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')

# Define Output format

In [2]:
class JSON_output(BaseModel):
    score: float = Field(description="score")
    comment: str = Field(description="comment")

parser = PydanticOutputParser(pydantic_object=JSON_output)

# Init llm

In [8]:
llm  = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4-1106-preview',
    temperature=0,
    max_tokens=512
)

In [81]:
# ReAct prompt
template = """You will be presented with an input answer and a ground truth in french. Your task is to score whether the input answer contains the information present in the ground truth or not.
    
    Approach this task step by step, take your time and do not skip any steps.

    1. Read the input answer and the ground truth.
    2. Information contained in the input answer might be explicited in a different format or phrasing. This doesn't affect the score.
    3. Determine and score whether the input answer contains the information within the ground truth based on the following rules:
       - If the input answer contains exactly the same information (facts, numbers, addresses, names, etc.) as the ground truth, assign a score of 1.
       - If the input answer contains exactly the same information as the ground truth plus additional information, then assign a score of 1
       - If the input answer only contains partial information present in the ground truth, then assign a score of 0.5
       - If the input answer doesn't contain any of the information in the ground truth, then assign a score of 0
    4. Output a response as JSON with keys as follows:
        - "score": allowable values are [0, 0.5, 1]
        - "comment": any suitable comment based on the analysis you performed, if required.

    Input answer: {input_answer}

    Ground truth: {ground_truth}
"""

prompt_template = PromptTemplate(input_variables=["input_answer", "ground_truth"], template=template)

# Preprocess eval data

In [57]:
with open('../evaluation/questions.json') as f:
    eval_set = json.load(f)

In [58]:
dfs = []
for q in eval_set["questions"]:

    df = pd.DataFrame.from_dict(q["reponse"])
    df["src"] = q["document"]
    dfs.append(df)
    
pd.concat(dfs).to_csv(f'../evaluation/eval_set.csv', index=None)

# Load eval data

In [59]:
eval_set = pd.read_csv("../evaluation/eval_set.csv")

# Compute eval score

In [69]:
import sys
sys.path.append("../scripts")

from rag import init_pinecone, init_vectorstore, init_retriever, init_retrievalqa_chain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [61]:
eval_set.head()

,question,reponse,src
0,Quelles sont les responsabilités du rectorat d...,Le rectorat veille à la réalisation des missio...,Statut-Version-1juillet2021.pdf
1,Comment sont désignés les membres de l'assembl...,La composition de l'assemblée de l'université ...,Statut-Version-1juillet2021.pdf
2,Quelles sont les conditions pour être considér...,Une personne est considérée comme étudiant ou ...,Statut-Version-1juillet2021.pdf
3,Quels sont les critères d'élimination d'un étu...,Un étudiant ou une étudiante est éliminé(e) d'...,Statut-Version-1juillet2021.pdf
4,Quelles sont les conditions pour l'exmatricula...,L'exmatriculation est demandée par l'étudiant ...,Statut-Version-1juillet2021.pdf


In [70]:
qa = init_retrievalqa_chain()

### Generate RAG answers

In [76]:
rag_answer = []

for question in tqdm.tqdm(eval_set["question"]):

    current_chat_history = ConversationBufferWindowMemory(
        memory_key='chat_history',
        input_key="question",
        k=3,
        return_messages=True
    )
    
    res = qa({"query": question, 
              "chat_history": current_chat_history})
    
    rag_answer.append(res["result"])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [05:18<00:00, 10.29s/it]


In [78]:
eval_set["rag_answer"] = rag_answer

In [79]:
k_docs = 5

eval_set.to_csv(f"../evaluation/evaluated/k_docs_{k_docs}.csv", index=None)

### Score answers

In [80]:
eval_set = pd.read_csv("../evaluation/evaluated/k_docs_5.csv")
eval_set.head()

,question,reponse,src,rag_answer
0,Quelles sont les responsabilités du rectorat d...,Le rectorat veille à la réalisation des missio...,Statut-Version-1juillet2021.pdf,Le rectorat de l'Université de Genève est resp...
1,Comment sont désignés les membres de l'assembl...,La composition de l'assemblée de l'université ...,Statut-Version-1juillet2021.pdf,Les membres de l'assemblée de l'université son...
2,Quelles sont les conditions pour être considér...,Une personne est considérée comme étudiant ou ...,Statut-Version-1juillet2021.pdf,Pour être considéré comme étudiant ou étudiant...
3,Quels sont les critères d'élimination d'un étu...,Un étudiant ou une étudiante est éliminé(e) d'...,Statut-Version-1juillet2021.pdf,Les critères d'élimination d'un étudiant ou d'...
4,Quelles sont les conditions pour l'exmatricula...,L'exmatriculation est demandée par l'étudiant ...,Statut-Version-1juillet2021.pdf,Les conditions pour l'exmatriculation d'un étu...


In [83]:
scores = {}
eval = []
for rag_answer, ground_truth in tqdm.tqdm(zip(eval_set["rag_answer"], eval_set["reponse"])):

    prompt = prompt_template.format(input_answer=rag_answer, ground_truth=ground_truth)
    res = llm([SystemMessage(content=prompt)])

    eval.append(parser.parse(res.content))

scores["k_docs_5"] = eval

31it [01:35,  3.06s/it]


In [87]:
eval_set["rag_answer"][0]

"Le rectorat de l'Université de Genève est responsable de surveiller et de coordonner l'application du règlement de l'université, en veillant à une interprétation et une application cohérentes dans chaque UPER/UER. Il peut déléguer certaines compétences à une commission ad hoc. Le rectorat est également l'autorité d'engagement pour les membres du corps des collaborateurs/trices de l'enseignement et de la recherche, bien que cette compétence soit déléguée au Décanat de l'UPER et à la direction de l'UER concernées pour certaines fonctions. En outre, le rectorat prend les décisions relatives au personnel conformément aux lois cantonales et sollicite l'avis du Décanat de l'UPER concernée avant de prendre des décisions. Il est aussi impliqué dans les procédures de plainte en cas de violation de la règle de préférence dans les nominations."

In [88]:
eval_set["reponse"][0]

"Le rectorat veille à la réalisation des missions et objectifs de l'université, au respect des valeurs académiques et éthiques, et à la réputation de l'université. La composition et le mode de désignation du rectorat sont régis par les articles 27 à 29 de la Loi sur l'université. Le recteur ou la rectrice est désigné(e) par l'assemblée de l'université après consultation du conseil d'orientation stratégique et nommé(e) par le Conseil d'État."

In [86]:
[x.score for x in scores["k_docs_5"]]

[0.0,
 0.0,
 0.5,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 1.0,
 1.0,
 0.5,
 0.0,
 1.0,
 1.0,
 0.5,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [92]:
print(eval_set["rag_answer"].iloc[-1])

Les conditions pour l'élimination d'un étudiant du titre qu'il brigue à l'Université de Genève sont les suivantes :

1. L'étudiant est éliminé s'il a répété sans succès l'année propédeutique.
2. L'étudiant est éliminé s'il ne peut plus répéter l’évaluation d’un enseignement des études de base, d’un certificat complémentaire, d’un certificat de spécialisation, d’un complément de bachelor ou du master of advanced studies (MAS) ou d’un certificat ou diplôme de formation continue non réussie selon l'article 14.
3. L'étudiant est éliminé s'il n’a pas obtenu au moins 20 crédits ECTS (équivalences exclues) durant les 2 premiers semestres de ses études de bachelor ou de master.
4. L'étudiant est éliminé s'il ne peut plus répéter l’évaluation d’un enseignement à option dans le cadre des limites fixées par l'article 9, alinéa 3.
5. L'étudiant est éliminé s'il n'a pas obtenu le titre brigué dans les délais d'études suivants, qui varient en fonction du nombre de crédits ECTS du titre :
   - 2 seme

In [91]:
eval_set["reponse"].iloc[-1]

"Un étudiant est éliminé du titre brigué s'il répète sans succès l'année propédeutique, ne peut répéter l’évaluation d’un enseignement non réussi, n'obtient pas au moins 20 crédits ECTS durant les 2 premiers semestres, ne peut répéter l’évaluation d’un enseignement à option, ou n'obtient pas le titre brigué dans les délais d'études fixés."